In [ ]:
%matplotlib inline
import tensorflow as tf
from tensorflow import keras
import numpy as np
import plot_utils
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython import display
print('Tensorflow version:', tf.__version__)

In [ ]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.fashion_mnist.load_data()
x_train=x_train.astype(np.float32)/255.0
x_test=x_test.astype(np.float32)/255.0

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.grid(False)
    plt.imshow(x_train[i],cmap=plt.cm.binary)

In [ ]:
batch_size=32
dataset=tf.data.Dataset.from_tensor_slices(x_train).shuffle(1000)
dataset=dataset.batch(batch_size,drop_remainder=True).prefetch(1)

In [ ]:
num_features=100

generator=keras.models.Sequential([
    keras.layers.Dense(7*7*128,input_shape=[num_features]),
    keras.layers.Reshape([7,7,128]),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(64,(5,5),(2,2),padding='same',activation='selu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(1,(5,5),(2,2),padding='same',activation='tanh'),
])

In [ ]:
discriminator=keras.models.Sequential([
    keras.layers.Conv2D(64,(5,5),(2,2),padding='same',input_shape=[28,28,1]),
    keras.layers.LeakyReLU(0.2),
    keras.layers.Dropout(0.3),
    keras.layers.Conv2D(64,(5,5),(2,2),padding='same'),
    keras.layers.LeakyReLU(0.2),
    keras.layers.Dropout(0.3),
    keras.layers.Flatten(),
    keras.layers.Dense(1,activation='sigmoid'),
    
])

In [ ]:
discriminator.compile(loss='binary_crossentropy',optimizer='rmsprop')
discriminator.trainable=False
gan=keras.models.Sequential([generator,discriminator])
gan.compile(loss='binary_crossentropy',optimizer='rmsprop')

In [ ]:
seed=tf.random.normal(shape=[batch_size,100])

In [ ]:
def train_dcgan(gan,dataset,batch_size,num_features,epochs=5):
    generator,discriminator=gan.layers
    for epoch in range(epochs):
        for X_batch in dataset:
            noise=tf.random.normal(shape=[batch_size,num_features])
            generated_images=generator(noise)
            X_fake_and_real=tf.concat([generated_images,X_batch],axis=0)
            y1=tf.constant([[0.]]*batch_size+[[1.]]*batch_size)
            discriminator.trainable=True
            discriminator.train_on_batch(X_fake_and_real,y1)
            y2=tf.constant([[1.]]*batch_size)
            discriminator.trainable=False
            gan.train_on_batch(noise,y2)

In [ ]:
x_train_dcgan=x_train.reshape(-1,28,28,1)*2.-1.

In [ ]:
batch_size=32
dataset=tf.data.Dataset.from_tensor_slices(x_train_dcgan).shuffle(1000)
dataset=dataset.batch(batch_size,drop_remainder=True).prefetch(1)

In [ ]:
train_dcgan(gan,dataset,batch_size,num_features,epochs=100)

In [ ]:
noise=tf.random.normal(shape=[batch_size,num_features])
generated_images=generator(noise)
plot_utils.show(generated_images,8)